In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\ds\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\DELL\\ds'

In [5]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config

In [8]:
import os 
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
    def transform(self):
        df = pd.read_csv(self.config.data_path)
        X = df.drop(columns=LABEL_COLUMN)
        y = df[LABEL_COLUMN]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        logger.info(f"X_train shape: {X_train.shape}")
        logger.info(f"X_test shape: {X_test.shape}")
        logger.info(f"y_train shape: {y_train.shape}")
        logger.info(f"y_test shape: {y_test.shape}")
        return X_train, X_test, y_train, y_test
    def save(self):
        pd.DataFrame(self.transform()).to_csv(self.config.data_path)
        logger.info(f"Data transformation saved to {self.config.data_path}")
    def load(self):
        df = pd.read_csv(self.config.data_path)
        return df


    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)
        # split the data
        train, test = train_test_split(data)
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False, header=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False, header=False)

    

        logger.info(f"Data splitted in {self.config.root_dir}")
        logger.info(f"Train: {os.path.join(self.config.root_dir, 'train.csv')}")
        logger.info(f"Test: {os.path.join(self.config.root_dir, 'test.csv')}")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()

except Exception as e:
    raise e

[2024-03-09 12:05:23,841: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-09 12:05:23,849: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-09 12:05:23,853: INFO: common: created directory at: artifacts]
[2024-03-09 12:05:23,857: INFO: common: created directory at: artifacts/data_transformation]
[2024-03-09 12:05:23,953: INFO: 693874325: Data splitted in artifacts/data_transformation]
[2024-03-09 12:05:23,956: INFO: 693874325: Train: artifacts/data_transformation\train.csv]
[2024-03-09 12:05:23,959: INFO: 693874325: Test: artifacts/data_transformation\test.csv]
[2024-03-09 12:05:23,962: INFO: 693874325: (1199, 12)]
[2024-03-09 12:05:23,965: INFO: 693874325: (400, 12)]
(1199, 12)
(400, 12)
